# Document Retrieval (40 pts)

Information Retrieval is the process of obtaining relevant information from a system based on a user’s query. One common example is retrieving documents that match a query from a large corpus. In the vector space model, documents are represented as vectors, and a query can also be represented as a vector. To find documents that are relevant to the query, we can measure the similarity between the query vector and the document vectors, using methods such as cosine similarity: <br><br>

$$cosine\,similarity = cos(\theta) = \frac{q \cdot d}{\|q\| \|d\|}$$

where **q** and **d** are the query and document vectors respectively.
How are these vector space models built? Let us represent a corpus of documents as a term-document matrix. In the term-document matrix, rows correspond to documents in the corpus and
columns correspond to terms(words). The entries in the matrix can be defined in different ways (we will describe
a few variations in this assignment question). A term document matrix allows for a way to index several
documents against which a user query can be compared to fetch relevant documents.  

Consider a corpus of documents:  

In [3]:
corpus = [
    "THE CAT IS ON THE MAT",
    "A DOG IS IN THAT GARDEN",
    "HE LIVES IN THE HOUSE WITH A GARDEN"
]

The term-document matrix of this corpus would be seem like this:

|        | THE | CAT | IS | ON | MAT | A | DOG | IN | THAT | GARDEN | HE | LIVES | HOUSE | WITH |
|--------|-----|-----|----|----|-----|---|-----|----|------|--------|----|-------|-------|------|
|  doc1  |     |     |    |    |     |   |     |    |      |        |    |       |       |      |      
|  doc2  |     |     |    |    |     |   |     |    |      |        |    |       |       |      |
|  doc3  |     |     |    |    |     |   |     |    |      |        |    |       |       |      |

## Cautions
* Implement the solution **without** importing additional libraries.
* Do **not** modify any part of the code that is not explicitly marked with “YOUR CODE.”

In [4]:
import numpy as np
import scipy as sp
import pandas as pd

## Problem 1. Simple Word Counts (15 pts)
In this problem, we will fill the entries of the term-document matrix using simple word counts.

### (a) Complete the <code>simple_word_counts</code> function. (5 pts)

**Instructions**

The function returns a term-document matrix for a given corpus using simple word counts (the number of times a word appears in a document).

In [ ]:
def simple_word_counts(corpus: list[str]) -> np.ndarray:
    
    term_doc_matrix = None

    ###############Your Code################
    # The order of words in the term-document matrix DOES NOT matter
    # Replace "pass" statement with your code
    vocabulary = sorted(set(word.lower() for doc in corpus for word in doc.split()))
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}

    term_doc_matrix = np.zeros((len(corpus), len(vocabulary)), dtype=int)

    for doc_idx, doc in enumerate(corpus):
        for word in doc.split(): 
            word = word.lower()
            if word in word2idx:
                term_doc_matrix[doc_idx, word2idx[word]] += 1
    ########################################

    return term_doc_matrix


term_doc_matrix = simple_word_counts(corpus)
term_doc_matrix


array([[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 2, 0],
       [1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1]])

### (b) Complete the <code>compute_sim</code> function. (5 pts)

**Instructions**

Using the simple word counts, the function computes the cosine similarity between a user query and each document in a given corpus.
The function returns the similarity scores for all documents in the corpus based on the query.

In [12]:
def cos_distance(a: np.ndarray, b: np.ndarray) -> float:
    return 1 - sp.spatial.distance.cosine(a, b)


def compute_sim(corpus: list[str], query: str) -> np.ndarray:
    
    cos_sim = None
    
    ###############Your Code################
    # Make use of cos_distance function above!
    # The result would be a 1D ndarray with shape (3, )
    # Replace "pass" statement with your code    
    query_words = query.lower().split()
    term_doc_matrix = simple_word_counts(corpus)
    vocabulary = sorted(set(word.lower() for doc in corpus for word in doc.split()))
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}

    query_vector = np.zeros(term_doc_matrix.shape[1], dtype=int)
    for word in query_words:
        if word in vocabulary:
            idx = word2idx[word]
            query_vector[idx] += 1
            
    cos_sim = np.zeros(len(corpus), dtype=float)
    for i in range(len(corpus)):
        cos_sim[i] = cos_distance(term_doc_matrix[i], query_vector)
    cos_sim = np.array(cos_sim, dtype=float)  # Ensure the result is a numpy array
    
    ########################################
    
    return cos_sim

query = "THE DOG LIVES IN THE GARDEN"
compute_sim(corpus, query)

array([0.5      , 0.4330127, 0.625    ])

### (c) Discuss the limitations of using simple word counts to create document vectors. (5 pts)


Answer : 
1. 관사 등의 카운트가 포함됨.
2. 문서가 길면 유사도가 높아짐.
3. 유의어들을 다 따로 카운트함.
4. 전체 집합이 커지면 문서 대부분의 값이 0으로 수렴함
5. 문맥을 반영하지 못 함.

## Problem 2. TF-IDF (25 pts)

In this section, you will explore Term Frequency-Inverse Document Frequency (TF-IDF) as an improvement over simple word counts.

TF-IDF involves multiplying the term frequency (which can be the simple word counts) by the inverse document frequency (IDF). The IDF measures how informative a word is, indicating whether it is common or rare across the entire corpus. It is calculated as the logarithm of the inverse of the fraction of documents that contain the word (i.e., the total number of documents divided by the number of documents in which the word appears, followed by taking the logarithm of this ratio).
<br><br>

$$idf(t) = log_2 \frac{(Number\,\,of\,\,documents\,\,in\,\,the\,\,corpus)}{(Number\,\,of\,\,documents\,\,t\,\,appears\,\,in)}$$  
  <br>
  
$$tf-idf \,\,=\,\, tf \times idf$$
<br>

### (a) Complete the <code>idf</code> function. (5 pts)

The function returns a IDF for a given corpus.

In [14]:
def idf(corpus: list[str]) -> np.ndarray:
    
    idf_doc = None

    ###############Your Code################
    # NOTE the base of log is 2, not 10 or e
    # The result would be a 1D ndarray with shape (14, )
    # Replace "pass" statement with your code
    vocabulary = sorted(set(word.lower() for doc in corpus for word in doc.split()))
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}
    term_doc_matrix = simple_word_counts(corpus)

    idf_doc = np.zeros(len(vocabulary), dtype=float)
    num_docs = len(corpus)

    for word, idx in word2idx.items():
        doc_freq = np.sum(term_doc_matrix[:, idx] > 0)
        if doc_freq > 0:
            idf_doc[idx] = np.log2(num_docs / doc_freq)
        else:
            idf_doc[idx] = 0.0
    idf_doc = np.array(idf_doc, dtype=float)  # Ensure the result is a numpy array
    
    ########################################

    return idf_doc


idf_val = idf(corpus)
idf_val

array([0.5849625, 1.5849625, 1.5849625, 0.5849625, 1.5849625, 1.5849625,
       0.5849625, 0.5849625, 1.5849625, 1.5849625, 1.5849625, 1.5849625,
       0.5849625, 1.5849625])

In [15]:
tf_idf_matrix = term_doc_matrix * idf_val
tf_idf_matrix

array([[0.       , 1.5849625, 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.5849625, 0.       , 1.5849625, 1.5849625, 0.       ,
        1.169925 , 0.       ],
       [0.5849625, 0.       , 1.5849625, 0.5849625, 0.       , 0.       ,
        0.5849625, 0.5849625, 0.       , 0.       , 0.       , 1.5849625,
        0.       , 0.       ],
       [0.5849625, 0.       , 0.       , 0.5849625, 1.5849625, 1.5849625,
        0.5849625, 0.       , 1.5849625, 0.       , 0.       , 0.       ,
        0.5849625, 1.5849625]])

### (b) Complete the <code>compute_sim_tfidf</code> function. (5 pts)


Using TF-IDF, the function computes the cosine similarity between a user query and each document in a given corpus.
The function returns the similarity scores for all documents in the corpus based on the query.

In [17]:
def compute_sim_tfidf(corpus: list[str], query: str) -> np.ndarray:
    
    cos_sim = None

    ###############Your Code################
    # The result would be a 1D ndarray with shape (3, )    
    # Replace "pass" statement with your code
    query_words = query.lower().split()
    term_doc_matrix = simple_word_counts(corpus)
    idf_val = idf(corpus)
    vocabulary = sorted(set(word.lower() for doc in corpus for word in doc.split()))
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}

    query_vector = np.zeros(term_doc_matrix.shape[1], dtype=float)
    for word in query_words:
        if word in vocabulary:
            idx = word2idx[word]
            query_vector[idx] += 1
    
    query_vector *= idf_val  # Apply IDF to the query vector
    cos_sim = np.zeros(len(corpus), dtype=float)
    for i in range(len(corpus)):
        cos_sim[i] = cos_distance(tf_idf_matrix[i], query_vector)
    cos_sim = np.array(cos_sim, dtype=float)  # Ensure the result is a numpy array
        
    ########################################
    
    return cos_sim


compute_sim_tfidf(corpus, query)

array([0.16919074, 0.47521095, 0.43172986])

### (c) Explain how the TF-IDF representation addresses the issues identified in Problem 1(c). (5 pts)

Answer :
1. 관사 등의 의미 없는 흔한 단어의 IDF 값이 작게 계산되어 영향이 줄어듦.
2. 상대 빈도를 계산하기 때문에 문서의 길이에 비교적 자유로움.
3. 드물게 등장하는 단어의 계산값이 높아져 정보량이 높은 단어를 강조함.

### (d) Complete the <code>compute_sim_tfidf_L2</code> function. (5 pts)

Instead of cosine similarity, the function computes the L2 distance between a user query and each document using TF-IDF.
The function returns the L2 distances for all documents in the corpus based on the query.

In [18]:
def compute_sim_tfidf_l2(corpus: list[str], query: str) -> np.ndarray:
    
    l2_dist = None

    ###############Your Code################
    # The result would be a 1D ndarray with shape (3, )
    # Replace "pass" statement with your code
    query_words = query.lower().split()
    term_doc_matrix = simple_word_counts(corpus)
    idf_val = idf(corpus)
    vocabulary = sorted(set(word.lower() for doc in corpus for word in doc.split()))
    word2idx = {word: idx for idx, word in enumerate(vocabulary)}
    query_vector = np.zeros(term_doc_matrix.shape[1], dtype=float)

    for word in query_words:
        if word in vocabulary:
            idx = word2idx[word]
            query_vector[idx] += 1
    query_vector *= idf_val  # Apply IDF to the query vector
    query_vector /= np.linalg.norm(query_vector)  # Normalize the query vector

    tf_idf_matrix = term_doc_matrix * idf_val

    l2_dist = np.zeros(len(corpus), dtype=float)
    for i in range(len(corpus)):
        doc_vector = tf_idf_matrix[i]
        doc_vector /= np.linalg.norm(doc_vector)
        l2_dist[i] = np.linalg.norm(doc_vector - query_vector)
    l2_dist = np.array(l2_dist, dtype=float)  # Ensure the result is a numpy array 
    ########################################
    
    return l2_dist


compute_sim_tfidf_l2(corpus, query)

array([1.28903783, 1.02448919, 1.06608644])

### (e) Compare the performance of cosine similarity and L2 distance in this task. Which one do you think works better and why? (5 pts)

Answer :
I would say cosine similarity work better than L2 distance. The given query is more about 2nd and 3rd doc in corpus and cosine similarity represents it well. On the other hand, L2 distance says that 1st doc is much similar with query than others which is wrong.